In [2]:
import numpy as np
from sortedcontainers import SortedList
import heapq
import itertools
import matplotlib.pyplot as plt  # Optional: For visualizing paths

def bounding_boxes_intersect(p1, p2):
    """
    Check if the bounding boxes of two sets of points overlap.

    Parameters
    ----------
    p1 : np.ndarray of shape (N, 2)
        First path points.
    p2 : np.ndarray of shape (M, 2)
        Second path points.

    Returns
    -------
    bool
        True if their bounding boxes overlap, False otherwise.
    """
    min_x_1, max_x_1 = np.min(p1[:, 0]), np.max(p1[:, 0])
    min_y_1, max_y_1 = np.min(p1[:, 1]), np.max(p1[:, 1])

    min_x_2, max_x_2 = np.min(p2[:, 0]), np.max(p2[:, 0])
    min_y_2, max_y_2 = np.min(p2[:, 1]), np.max(p2[:, 1])

    overlap_x = not (max_x_1 < min_x_2 or max_x_2 < min_x_1)
    overlap_y = not (max_y_1 < min_y_2 or max_y_2 < min_y_1)
    return overlap_x and overlap_y

def paths_intersect(p1, p2, stop_at_first=True):
    """
    Sweep-line algorithm (Bentley-Ottmann style) to check if two polygonal paths intersect.
    This version includes unique identifiers to prevent comparison errors.

    Parameters
    ----------
    p1 : np.ndarray of shape (N, 2)
        Points of the first path.
    p2 : np.ndarray of shape (M, 2)
        Points of the second path.
    stop_at_first : bool, optional
        If True, return as soon as any intersection is found. Default is True.

    Returns
    -------
    bool
        True if the paths intersect, False otherwise.
    """
    # Quick bounding box check
    if not bounding_boxes_intersect(p1, p2):
        return False

    # Initialize unique counters for segments and events
    segment_id_counter = itertools.count()
    event_id_counter = itertools.count()

    # Define the Segment class with unique identifiers
    class Segment:
        __slots__ = ['p1', 'p2', 'idx']

        def __init__(self, p1, p2, idx):
            if p2[0] < p1[0]:
                p1, p2 = p2, p1
            self.p1 = p1
            self.p2 = p2
            self.idx = idx  # Unique identifier

        def y_at_x(self, x):
            """
            Compute the y-coordinate of the intersection of this segment
            with a vertical line at x.

            Parameters
            ----------
            x : float
                The x-coordinate at which to compute y.

            Returns
            -------
            float
                The y-coordinate of the segment at x.
            """
            if abs(self.p2[0] - self.p1[0]) < 1e-14:
                return self.p1[1]  # Vertical line, y is constant
            t = (x - self.p1[0]) / (self.p2[0] - self.p1[0])
            return self.p1[1] + t * (self.p2[1] - self.p1[1])

    # Helper functions
    def orientation(p, q, r, eps=1e-14):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if abs(val) < eps:
            return 0  # Collinear
        return 1 if val > 0 else 2  # Clockwise or Counterclockwise

    def on_segment(p, q, r, eps=1e-14):
        return (min(p[0], r[0]) - eps <= q[0] <= max(p[0], r[0]) + eps and
                min(p[1], r[1]) - eps <= q[1] <= max(p[1], r[1]) + eps)

    def segments_intersect(s1_start, s1_end, s2_start, s2_end, eps=1e-14):
        o1 = orientation(s1_start, s1_end, s2_start, eps)
        o2 = orientation(s1_start, s1_end, s2_end, eps)
        o3 = orientation(s2_start, s2_end, s1_start, eps)
        o4 = orientation(s2_start, s2_end, s1_end, eps)

        # General case
        if o1 != o2 and o3 != o4:
            return True

        # Special cases
        if o1 == 0 and on_segment(s1_start, s2_start, s1_end, eps):
            return True
        if o2 == 0 and on_segment(s1_start, s2_end, s1_end, eps):
            return True
        if o3 == 0 and on_segment(s2_start, s1_start, s2_end, eps):
            return True
        if o4 == 0 and on_segment(s2_start, s1_end, s2_end, eps):
            return True

        return False

    def segment_intersection_point(s1_start, s1_end, s2_start, s2_end, eps=1e-14):
        denom = ((s1_end[0] - s1_start[0]) * (s2_end[1] - s2_start[1]) -
                 (s1_end[1] - s1_start[1]) * (s2_end[0] - s2_start[0]))
        if abs(denom) < eps:
            return None  # Parallel or nearly parallel

        t1 = ((s1_start[0] - s2_start[0]) * (s2_end[1] - s2_start[1]) -
              (s1_start[1] - s2_start[1]) * (s2_end[0] - s2_start[0])) / denom

        ix = s1_start[0] + t1 * (s1_end[0] - s1_start[0])
        iy = s1_start[1] + t1 * (s1_end[1] - s1_start[1])

        inter_pt = (ix, iy)
        if segments_intersect(s1_start, s1_end, s2_start, s2_end, eps):
            return inter_pt
        else:
            return None

    # Build the segments from both paths
    segs = []
    for i in range(p1.shape[0] - 1):
        seg = Segment(tuple(p1[i]), tuple(p1[i+1]), next(segment_id_counter))
        segs.append(seg)
    for j in range(p2.shape[0] - 1):
        seg = Segment(tuple(p2[j]), tuple(p2[j+1]), next(segment_id_counter))
        segs.append(seg)

    # Initialize event queue with start and end events
    events = []
    for s in segs:
        heapq.heappush(events, (s.p1[0], 0, next(event_id_counter), s))  # START
        heapq.heappush(events, (s.p2[0], 1, next(event_id_counter), s))  # END

    # Initialize the active set
    active = SortedList(key=lambda seg: (round(seg.y_at_x(current_x), 14), seg.idx) if 'current_x' in locals() else (float('-inf'), seg.idx))

    # Helper functions within paths_intersect
    def get_active_index(segment):
        try:
            return active.index(segment)
        except ValueError:
            return None

    def neighbor_segments(idx):
        left = active[idx - 1] if idx - 1 >= 0 else None
        right = active[idx + 1] if idx + 1 < len(active) else None
        return left, right

    def check_and_add_intersection(s1, s2, current_x):
        if s1 is None or s2 is None or s1 == s2:
            return
        pt = segment_intersection_point(s1.p1, s1.p2, s2.p1, s2.p2)
        if pt is not None:
            ix, iy = pt
            if ix > current_x + 1e-14:
                heapq.heappush(events, (ix, 2, next(event_id_counter), (s1, s2, pt)))  # INTS

    # Process events
    while events:
        x, etype, eid, data = heapq.heappop(events)
        current_x = x

        # Rebuild active set ordering based on new current_x
        if 'current_x' in locals():
            old_segments = list(active)
            active.clear()
            active.update(old_segments)

        if etype == 0:  # START
            s = data
            active.add(s)
            idx = get_active_index(s)
            left_s, right_s = neighbor_segments(idx)
            check_and_add_intersection(left_s, s, current_x)
            check_and_add_intersection(s, right_s, current_x)

            if stop_at_first:
                # Check immediate neighbors for intersection
                if (left_s and segments_intersect(left_s.p1, left_s.p2, s.p1, s.p2)) or \
                   (right_s and segments_intersect(s.p1, s.p2, right_s.p1, right_s.p2)):
                    return True

        elif etype == 1:  # END
            s = data
            idx = get_active_index(s)
            if idx is not None:
                left_s, right_s = neighbor_segments(idx)
                active.remove(s)
                check_and_add_intersection(left_s, right_s, current_x)

                if stop_at_first and left_s and right_s and segments_intersect(left_s.p1, left_s.p2, right_s.p1, right_s.p2):
                    return True

        elif etype == 2:  # Intersection
            s1, s2, pt = data
            idx1 = get_active_index(s1)
            idx2 = get_active_index(s2)
            if idx1 is None or idx2 is None:
                continue  # One of the segments has been removed

            if idx1 > idx2:
                s1, s2 = s2, s1
                idx1, idx2 = idx2, idx1

            # Swap the segments in the active set
            active.remove(s1)
            active.remove(s2)
            active.add(s2)
            active.add(s1)

            # Check for new intersections
            new_idx1 = active.index(s2)
            new_idx2 = active.index(s1)

            left_s, _ = neighbor_segments(new_idx1)
            _, right_s = neighbor_segments(new_idx2)

            check_and_add_intersection(left_s, s2, current_x)
            check_and_add_intersection(s1, right_s, current_x)

            if stop_at_first:
                return True

    # If no intersections found
    return False


def generate_random_circle_chord(num_points=30, radius=1.0, jitter=0.0):
    """
    Generate a random chord path on a circle.

    Parameters
    ----------
    num_points : int
        Number of points defining the path.
    radius : float
        Radius of the circle.
    jitter : float
        Maximum random displacement added to each point to avoid perfect circles.

    Returns
    -------
    np.ndarray of shape (num_points, 2)
        The generated path points.
    """
    angles = np.linspace(0, 2 * np.pi, num_points, endpoint=False)
    angles += np.random.uniform(-np.pi / num_points, np.pi / num_points, size=num_points)  # Random perturbation
    x = radius * np.cos(angles) + np.random.uniform(-jitter, jitter, size=num_points)
    y = radius * np.sin(angles) + np.random.uniform(-jitter, jitter, size=num_points)
    return np.vstack((x, y)).T


# Number of path pairs to benchmark
num_pairs = 100

# Generate list of path pairs
path_pairs = []
for _ in range(num_pairs):
    p1 = generate_random_circle_chord(jitter=0.05)
    p2 = generate_random_circle_chord(jitter=0.05)
    path_pairs.append((p1, p2))

def benchmark_bounding_box(path_pairs):
    """
    Apply bounding_boxes_intersect to all path pairs.

    Parameters
    ----------
    path_pairs : list of tuples
        List containing tuples of (p1, p2).

    Returns
    -------
    list of bool
        Results of bounding_boxes_intersect for each pair.
    """
    results = []
    for p1, p2 in path_pairs:
        results.append(bounding_boxes_intersect(p1, p2))
    return results

def benchmark_full_intersection(path_pairs):
    """
    Apply paths_intersect to all path pairs.

    Parameters
    ----------
    path_pairs : list of tuples
        List containing tuples of (p1, p2).

    Returns
    -------
    list of bool
        Results of paths_intersect for each pair.
    """
    results = []
    for p1, p2 in path_pairs:
        results.append(paths_intersect(p1, p2))
    return results

In [3]:
# Benchmark Bounding Box Intersection
%timeit benchmark_bounding_box(path_pairs)

# Benchmark Full Sweep-Line Intersection
%timeit benchmark_full_intersection(path_pairs)

1.04 ms ± 18.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
10.7 ms ± 116 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
